## 1) Initialisation & Chargement des données

In [2]:
import pyspark
print(pyspark.__version__)

3.5.3


In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("Spark_Final_Project").master("local[*]").getOrCreate()

df_raw = spark.read.option("Header", True).option("inferSchema", True).option("delimiter", ";").csv("../data/raw/Online_Retail_CSV.csv")

df_raw.printSchema()
df_raw.show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01/12/2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01/12/2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01/12/2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNI

In [11]:
print(f"Il y a {df_raw.count()} lignes.")
print(f"Il y a {len(df_raw.columns)} colonnes.")

Il y a 541909 lignes.
Il y a 8 colonnes.


## 2) Exploration & Prétraitement

## A) Analyse Descriptive

Tout d'abord, regardons combien il y a de clients référencés dans cette base de données :

In [15]:
print(f"Il y a {df_raw.select('CustomerID').distinct().count()} clients.")

Il y a 4373 clients.


Maintenant, regardons le nombre total de transactions :

In [22]:
transaction_nb = df_raw.select('InvoiceNo').dropDuplicates().count()
print(f"Il y a {transaction_nb} transactions uniques.")

Il y a 25900 transactions uniques.


Regardons de plus près la distribution de la variable Quantity :

In [24]:
df_raw.select("Quantity").describe().show()

+-------+------------------+
|summary|          Quantity|
+-------+------------------+
|  count|            541909|
|   mean|  9.55224954743324|
| stddev|218.08115785023418|
|    min|            -80995|
|    max|             80995|
+-------+------------------+



Nous observons qu’en moyenne, une ligne de commande contient environ 9 produits.  
Cependant, l’écart-type est relativement élevé (218), ce qui indique une forte dispersion des valeurs et la présence d’outliers dans la variable Quantity.  
En effet, certaines transactions présentent des quantités extrêmement faibles ou élevées, allant jusqu’à –80 995 et +80 995 unités.  
Ces valeurs extrêmes contribuent fortement à l’augmentation de l’écart-type et sont susceptibles de correspondre à des retours de produits, des annulations de commandes ou des commandes en gros.

Regardons egalement la distribution de la variable UnitPrice :

In [50]:
df_raw.select("UnitPrice").describe().show()

+-------+------------------+
|summary|         UnitPrice|
+-------+------------------+
|  count|            541909|
|   mean|29.921163668665333|
| stddev| 595.7455525989117|
|    min|         -11062,06|
|    max|             99,96|
+-------+------------------+



Nous observons que le prix unitaire moyen d’un produit est d’environ 30 unités monétaires.  
Toutefois, l’écart-type est particulièrement élevé (596), ce qui traduit une forte dispersion des valeurs et la présence d’outliers dans la variable UnitPrice.  
En effet, certaines transactions présentent des valeurs négatives, pouvant atteindre –11 062, qui ne correspondent pas à des prix réels mais sont probablement liées à des remboursements, des annulations de commandes ou des écritures de correction comptable.  
Ces valeurs extrêmes contribuent fortement à l’augmentation de l’écart-type.

Maintenant, intéressons-nous au nombre de pays clients :

In [36]:
client_countries = df_raw.select("Country").dropDuplicates().count()
print(f"Il y a {client_countries} pays qui sont clients.")

Il y a 38 pays qui sont clients.


Regardons le nombre de commandes par pays : 

In [34]:
df_raw.select("InvoiceNo", "Country").dropDuplicates().groupby("Country").count().orderBy("count", ascending=False).show()

+---------------+-----+
|        Country|count|
+---------------+-----+
| United Kingdom|23494|
|        Germany|  603|
|         France|  461|
|           EIRE|  360|
|        Belgium|  119|
|          Spain|  105|
|    Netherlands|  101|
|    Switzerland|   74|
|       Portugal|   71|
|      Australia|   69|
|          Italy|   55|
|        Finland|   48|
|         Sweden|   46|
|         Norway|   40|
|Channel Islands|   33|
|          Japan|   28|
|         Poland|   24|
|        Denmark|   21|
|         Cyprus|   20|
|        Austria|   19|
+---------------+-----+
only showing top 20 rows



Nous observons que le jeu de données comprend des clients provenant de 38 pays différents, ce qui indique une base de clientèle internationale.  
Toutefois, une très grande proportion des transactions est réalisée au Royaume-Uni, avec plus de 22 000 factures uniques, tandis que le deuxième pays, l’Allemagne, ne compte que 603 transactions.  
Cela met en évidence un fort déséquilibre géographique, le Royaume-Uni dominant largement le jeu de données par rapport aux autres pays.

Regardons maintenant le nombre total de commandes par clients, cela nous permettra de distinguer les clients habituels des clients ponctuels : 

In [43]:
df_raw.select("CustomerID", "InvoiceNo").dropDuplicates().groupBy("CustomerID").count().orderBy("count", ascending=False).show()

+----------+-----+
|CustomerID|count|
+----------+-----+
|      NULL| 3710|
|     14911|  248|
|     12748|  224|
|     17841|  169|
|     14606|  128|
|     15311|  118|
|     13089|  118|
|     12971|   89|
|     14527|   86|
|     13408|   81|
|     14646|   77|
|     16029|   76|
|     16422|   75|
|     14156|   66|
|     13798|   63|
|     18102|   62|
|     13694|   60|
|     15061|   55|
|     17450|   55|
|     16013|   54|
+----------+-----+
only showing top 20 rows



On remarque la présence d’un groupe CustomerID = NULL avec un nombre élevé de commandes. Cette observation suggère la présence de transactions sans identification client, ce qui motive l’analyse des valeurs manquantes présentée dans la section suivante.

## B) Vérifications de valeurs manquantes

Avant toute analyse plus poussée, il est important de vérifier les valeurs nulles :  

In [40]:
from pyspark.sql.functions import col, sum, when

df_raw.select([sum(when(col(c).isNull(), 1 ).otherwise(0)).alias(c) for c in df_raw.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



Nous observons la présence de valeurs manquantes dans deux colonnes sur huit : Description et CustomerID.  
La colonne Description contient un nombre relativement limité de valeurs manquantes (1 454), ce qui reste marginal au regard de la taille du jeu de données et n’impactera pas significativement l’analyse, d’autant plus que cette variable ne sera pas utilisée directement dans les modèles.  
En revanche, la colonne CustomerID présente un nombre important de valeurs manquantes (135 080), ce qui constitue un enjeu majeur pour l’analyse, notamment pour la segmentation client. Ces lignes devront faire l’objet d’un traitement spécifique lors de la phase de nettoyage des données.

Pour pallier au problème de valeurs manquantes mais egalement de prix et de quantité négatives, nous allons créer un nouveau dataset filtré sur ces trois conditions :  
1 - Les valeurs de CustomerID doivent êtres Non Nulles  
2 - Les valeurs de Quantity doivent êtres > 0  
3 - Les valeurs unitaires des produits doivent êtres > 0  

Nous créons donc ce dataset :

In [48]:
df_cleanV1 = (
    df_raw
    .filter(col("CustomerID").isNotNull())
    .filter(col("Quantity") > 0)
    .filter(col("UnitPrice") > 0)
)

La colonne InvoiceDate est initialement stockée sous forme de chaîne de caractères.  
Afin de permettre les calculs temporels nécessaires à l’analyse RFM, cette variable est convertie en format timestamp.

In [55]:
from pyspark.sql.functions import to_timestamp, col

df_cleanV1 = df_cleanV1.withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), "dd/MM/yyyy HH:mm"))
df_cleanV1.printSchema()
df_cleanV1.select("InvoiceDate").show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+-------------------+
|        InvoiceDate|
+-------------------+
|2010-12-01 08:45:00|
|2010-12-01 10:29:00|
|2010-12-01 11:27:00|
|2010-12-01 13:04:00|
|2010-12-01 14:05:00|
+-------------------+
only showing top 5 rows



In [59]:
from pyspark.sql.functions import regexp_replace

df_cleanV1 = df_cleanV1.withColumn("UnitPrice", regexp_replace(col("UnitPrice"), ",", "." ).cast("double"))

df_cleanV1.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

